In [182]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time
from konlpy.tag import *
import numpy as np
import math

In [124]:
data = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/토큰화/token_total.xlsx")

In [125]:
#드라이버 경로 지정
path = 'c:/Users/HoonCheol/chromedriver.exe' ## 크롬드라이브가 설치된 경로 지정
driver = webdriver.Chrome(path)
basic_url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query='

In [149]:
#두개 붙인 복합명사 추가 함수
def get_complex(dataframe,tokenizer):
    data = dataframe
    tokener = tokenizer + "_token"
    token = data[tokener] # 한나눔 명사 토큰들
    questionss = []
    start = time.time()
    for i in token:
        questions = [] # 복합명사 리스트
        list_token = i.split() # 한 행의 명사 토큰들
        for j, noun in enumerate(list_token[:-1]):
            question = noun + list_token[j+1] #복합명사 만들기
            driver.get(basic_url+question)
            delay = 0.1
            driver.implicitly_wait(delay)
        
            search = driver.find_element_by_class_name("main_pack")
        
            #한글만
            onlyKorean = re.compile("[^ㄱ-ㅣ가-힣\s]+")
            korean = onlyKorean.sub('',search.get_attribute("innerHTML"))
        
            #검색 결과 존재하면 복합명사로 판단 파라미터
            if korean.count(question) - 11 > 3:
                questions.append(question)
        questionss.append(questions)
    end = time.time()
    print("It took ",end - start, "seconds.")
    for i in range(len(questionss)):
        questionss[i] = " ".join(questionss[i])
    tokener = tokenizer + "_complex"
    data[tokener] = questionss
    return data

In [150]:
#3개 붙인 복합명사 추가 함수
def get_complex_plus(dataframe,tokenizer):
    data = dataframe
    tokener = tokenizer + "_token"
    token = data[tokener] # 한나눔 명사 토큰들
    questionss = []
    start = time.time()
    for i in token:
        questions = [] # 복합명사 리스트
        list_token = i.split() # 한 행의 명사 토큰들
        for j, noun in enumerate(list_token[:-2]):
            question = noun + list_token[j+1] + list_token[j+2] #복합명사 만들기
            driver.get(basic_url+question)
            delay = 0.1
            driver.implicitly_wait(delay)
        
            search = driver.find_element_by_class_name("main_pack")
        
            #한글만
            onlyKorean = re.compile("[^ㄱ-ㅣ가-힣\s]+")
            korean = onlyKorean.sub('',search.get_attribute("innerHTML"))
        
            #검색 결과 존재하면 복합명사로 판단 파라미터
            if korean.count(question) - 11 > 3:
                questions.append(question)
        questionss.append(questions)
    end = time.time()
    print("It took ",end - start, "seconds.")
    for i in range(len(questionss)):
        questionss[i] = " ".join(questionss[i])
    tokener = tokenizer + "_complex_plus"
    data[tokener] = questionss
    return data

In [138]:
def get_denominator(dataframe):
    data = dataframe
    x = data["answer"]
    x = x.values
    x_num = []
    for i in range(len(x)):
        x_num.append(len(x[i].split()))
    data["answer_num"] = x_num
    return data

In [188]:
def get_valid_denominator(dataframe, tokenizer):
    data = dataframe
    tokener = tokenizer + "_complex"
    x = data[tokener]
    x.values
    x_num = []
    for i in range(len(x)):
        try:
            x_num.append(len(x[i].split()))
        except:
            x_num.append(0)
    tokener = tokenizer[:3] + "_complex_num"
    data[tokener] = x_num
    
    tokener = tokenizer + "_complex_plus"
    x = data[tokener]
    x.values
    x_num = []
    for i in range(len(x)):
        try:
            x_num.append(len(x[i].split()))
        except:
            x_num.append(0)
    tokener = tokenizer[:3] + "_plus_num"
    data[tokener] = x_num
    return data

In [140]:
def get_accuracy_origin(dataframe,tokenizer):
    data = dataframe
    answer = data["answer"]
    tokener = tokenizer + "_token"
    tokener = data[tokener]
    answer = answer.values
    tokener = tokener.values
    
    counts = []
    for i in range(len(answer)):
        a = answer[i].split()
        b = tokener[i].split()
        count = 0
        for i, word in enumerate(b):
            if word in a:
                count+=1
        counts.append(count)
    tokener = tokenizer[:3] + "_vaild_num_origin"
    data[tokener] = counts
    
    data[tokenizer[:3] + "_acc"] = data[tokener] / data["answer_num"]
    
    return data

In [141]:
def get_accuracy_complex(dataframe,tokenizer):
    data = dataframe
    answer = data["answer"]
    tokener = tokenizer + "_complex"
    tokener = data[tokener]
    answer = answer.values
    tokener = tokener.values
    
    counts = []
    for i in range(len(answer)):
        a = answer[i].split()
        b = tokener[i].split()
        count = 0
        for i, word in enumerate(b):
            if word in a:
                count+=1
        counts.append(count)
    tokener = tokenizer[:3] + "_vaild_num_complex"
    data[tokener] = counts
    
    data[tokenizer[:3] + "_acc_cpx"] = data[tokener] / data["answer_num"]
    
    return data

In [142]:
def get_accuracy_plus(dataframe,tokenizer):
    data = dataframe
    answer = data["answer"]
    tokener = tokenizer + "_complex_plus"
    tokener = data[tokener]
    answer = answer.values
    tokener = tokener.values
    
    counts = []
    for i in range(len(answer)):
        a = answer[i].split()
        b = tokener[i].split()
        count = 0
        for i, word in enumerate(b):
            if word in a:
                count+=1
        counts.append(count)
    tokener = tokenizer[:3] + "_valid_num_plus"
    data[tokener] = counts
    
    data[tokenizer[:3] + "_acc_plus"] = data[tokener] / data["answer_num"]
    
    return data

In [ ]:
data = get_complex(data, "hannanum")

In [ ]:
data = get_complex(data, "twitter")

In [ ]:
data = 

In [143]:
get_complex(data, "komoran")

KeyboardInterrupt: 

In [52]:
data = get_complex_plus(data, "hannanum")
data = get_complex_plus(data, "twitter")
data = get_complex_plus(data, "komoran")

It took  253.15326738357544 seconds.
It took  418.87864995002747 seconds.
It took  387.05599641799927 seconds.


In [53]:
data = get_denominator(data)

In [74]:
data = get_accuracy_origin(data, "hannanum")
data = get_accuracy_origin(data, "twitter")
data = get_accuracy_origin(data, "komoran")

In [75]:
data = get_accuracy_complex(data, "hannanum")
data = get_accuracy_complex(data, "twitter")
data = get_accuracy_complex(data, "komoran")

In [76]:
data = get_accuracy_plus(data, "hannanum")
data = get_accuracy_plus(data, "twitter")
data = get_accuracy_plus(data, "komoran")

In [59]:
data

,질문,answer,twitter_token,hannanum_token,komoran_token,hannanum_complex,twitter_complex,komoran_complex,hannanum_complex_plus,twitter_complex_plus,...,twi_num,twi_acc,kom_num,kom_acc,han_acc_cpx,twi_acc_cpx,kom_acc_cpx,han_acc_plus,twi_acc_plus,kom_acc_plus
0,(계약자=수익자) 수익자가 법정상속인이 아닌 경우 계약자 사망으로 상속포기시 사망보...,계약자 수익자 수익자 법정상속인 아닌경우 계약자 사망 상속포기 사망보험금 수령 관계,계약 수익 수익 법정상속 경우 계약 사망 상속 포기 시 사망 보험금 수령 관계,(계약자=수익자) 수익자 법정상속인 경우 계약자 사망 상속포기시 사망보험금 수령 관계,계약자 수익자 수익자 법정 상속인 경우 계약자 사망 상속 포기 사망 보험금 수령 관계,,상속포기 포기시 시사망 사망보험금 보험금수령,법정상속인 상속포기 사망보험금 보험금수령,,상속포기시,...,0,0.363636,0,0.727273,0.000000,0.181818,0.272727,0.0,0.000000,0.000000
1,가업상속공제 요건 및 사후관리,가업상속공제 요건 사후관리,가업 상속 공제 요건 및 사후관리,가업상속공제 요 사후관리,가업 상속 공제 요건 사후 관리,,가업상속 상속공제 공제요건 요건및 및사후관리,가업상속 상속공제 공제요건 사후관리,,가업상속공제,...,1,0.666667,1,0.333333,0.000000,0.000000,0.333333,0.0,0.333333,0.333333
2,"가업승계, 가업상속과 관련된 조세 지원제도",가업승계 가업상속 관련 조세 지원제도,가업승계 가업 상속 관련 조세 지원 제도,가업승계 가업상속 관련 조세 지원제도,가업 승계 가업 상속 관련 조세 지원 제도,조세지원제도,가업상속 상속관련 조세지원 지원제도,가업승계 가업상속 상속관련 조세지원 지원제도,,조세지원제도,...,0,0.600000,0,0.400000,0.000000,0.400000,0.600000,0.0,0.000000,0.000000
3,가업승계주식 증여특례 요건 및 사후관리,가업승계주식 증여특례 요건 사후관리,가업승계 주식 증여 특례 요건 및 사후관리,가업승계주식 증여특례 요 사후관리,가업 승계 주식 증여 특례 요건 사후 관리,,주식증여 증여특례 특례요건 요건및 및사후관리,가업승계 주식증여 증여특례 특례요건 사후관리,,,...,0,0.500000,0,0.250000,0.000000,0.250000,0.500000,0.0,0.000000,0.000000
4,"가족, 자녀 명의의 금융거래도 차명거래 금지 대상 여부",가족명의 자녀명의 금융거래 차명거래 금지대상여부,가족 자녀 명의 금융 거래 차 거래 금지 대상 여부,가족 자녀 명의 금융거래 차명거래 금지 대상 여부,가족 자녀 명의 금융 거래 차명 거래 금지 대상 여부,자녀명의 차명거래금지 금지대상 대상여부,자녀명의 금융거래 거래차 차거래 거래금지 금지대상 대상여부,자녀명의 금융거래 차명거래 거래금지 금지대상 대상여부,,,...,0,0.000000,0,0.000000,0.200000,0.400000,0.600000,0.0,0.000000,0.000000
5,간주 상속재산이란,간주상속재산,간주 상속 재산,간주 상속재산,간주 상속 재산,간주상속재산,간주상속 상속재산,간주상속 상속재산,,간주상속재산,...,1,0.000000,1,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,1.000000
6,계약자(보험료 납입자)와 수익자(보험금 수령자)가 다른 경우 계약자가 중도인출을 하...,계약자 수익자 다른경우 계약자 중도인출,계약 보험료 납 입자 수익 보험금 수령 다른 경우 계약 중도,계약자(보험료 납입자 수익자(보험금 수령자 경우 계약자 중도인출,계약자 보험료 납 입자 수익자 보험금 수령자 경우 계약자 중도 인출,,보험료납 납입자 보험금수령,보험료납 납입자 중도인출,,,...,0,0.000000,0,0.600000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000
7,계약자(보험료 납입자)와 수익자(보험금 수령자)가 다른 경우 수익자가 보험금을 수령...,계약자 수익자 다른경우 수익자 보험금수령 증여세 어떻 계산,계약 보험료 납 입자 수익 보험금 수령 다른 경우 수익 보험금 수령 증여세 계산,계약자(보험료 납입자 수익자(보험금 수령자 경우 수익자 보험금 수령 증여세 계산,계약자 보험료 납 입자 수익자 보험금 수령자 경우 수익자 보험금 수령 증여세 계산,보험금수령,보험료납 납입자 보험금수령 보험금수령,보험료납 납입자 보험금수령,,,...,0,0.250000,0,0.625000,0.125000,0.250000,0.125000,0.0,0.000000,0.000000
8,계약자(보험료 납입자)와 수익자(보험금 수령자)가 다른 경우 증여세는 언제 부과되나요?,계약자 수익자 다른경우 증여세 언제부과,계약 보험료 납 입자 수익 보험금 수령 다른 경우 증여세 언제 부과,계약자(보험료 납입자 수익자(보험금 수령자 경우 증여세 부과,계약자 보험료 납 입자 수익자 보험금 수령자 경우 증여세 부과,증여세부과,보험료납 납입자 보험금수령,보험료납 납입자 증여세부과,,,...,0,0.200000,0,0.600000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,계약자에게 해지할 권한이 없는 경우 상속재산평가,계약자 해지권한 없는경우 상속재산평가,계약 해지 권한 경우 상속 재산 평가,계약자 해 권한 경우 상속재산평가,계약자 해지 권한 경우 상속 재산 평가,,계약해지 상속재산 재산평가,상속재산 재산평가,,,...,0,0.000000,0,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [78]:
data.to_csv("first.csv", encoding="utf-8-sig", index = False)
data.to_excel("first.xlsx", index = False)

In [73]:
data.columns

Index(['질문', 'answer', 'twitter_token', 'hannanum_token', 'komoran_token',
       'hannanum_complex', 'twitter_complex', 'komoran_complex',
       'hannanum_complex_plus', 'twitter_complex_plus', 'komoran_complex_plus',
       'answer_num'],
      dtype='object')

In [79]:
data = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/토큰화/token3.xlsx")

In [80]:
data = get_complex(data, "hannanum")
data = get_complex(data, "twitter")
data = get_complex(data, "komoran")

It took  126.48382544517517 seconds.
It took  153.9105350971222 seconds.
It took  114.92675399780273 seconds.


In [85]:
data = get_complex_plus(data, "hannanum")
data = get_complex_plus(data, "twitter")
data = get_complex_plus(data, "komoran")

It took  68.9815845489502 seconds.
It took  105.92482018470764 seconds.
It took  83.59950423240662 seconds.


In [86]:
data = get_denominator(data)

In [87]:
data = get_accuracy_origin(data, "hannanum")
data = get_accuracy_origin(data, "twitter")
data = get_accuracy_origin(data, "komoran")

In [88]:
data = get_accuracy_complex(data, "hannanum")
data = get_accuracy_complex(data, "twitter")
data = get_accuracy_complex(data, "komoran")

In [89]:
data = get_accuracy_plus(data, "hannanum")
data = get_accuracy_plus(data, "twitter")
data = get_accuracy_plus(data, "komoran")

In [91]:
data.to_csv("second.csv", encoding = 'utf-8-sig', index = False)
data.to_excel("second.xlsx", index = False)

In [92]:
data = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/토큰화/token7.xlsx")

In [93]:
data = get_complex(data, "hannanum")
data = get_complex(data, "twitter")
data = get_complex(data, "komoran")

data = get_complex_plus(data, "hannanum")
data = get_complex_plus(data, "twitter")
data = get_complex_plus(data, "komoran")

data = get_denominator(data)

data = get_accuracy_origin(data, "hannanum")
data = get_accuracy_origin(data, "twitter")
data = get_accuracy_origin(data, "komoran")

data = get_accuracy_complex(data, "hannanum")
data = get_accuracy_complex(data, "twitter")
data = get_accuracy_complex(data, "komoran")

data = get_accuracy_plus(data, "hannanum")
data = get_accuracy_plus(data, "twitter")
data = get_accuracy_plus(data, "komoran")

data.to_csv("third.csv", encoding = 'utf-8-sig', index = False)
data.to_excel("third.xlsx", index = False)

It took  95.09576892852783 seconds.
It took  152.71373343467712 seconds.
It took  116.65712809562683 seconds.
It took  57.997949838638306 seconds.
It took  114.49790096282959 seconds.
It took  84.57689166069031 seconds.


# 급히 뉴데이터 다듬은것

In [120]:
data = pd.read_excel("new_questions.xlsx")

In [110]:
data

,질문,answer
0,수익자가 법정상속인이 아닌 경우 계약자의 사망으로 인해 상속을 포기하면 사망보험금 ...,수익자 법정상속인 아닌경우 계약자 사망 상속포기 사망보험금 수령 관계
1,가업상속공제 요건과 사후관리를 어떻게 하는지 알고 싶습니다.,가업상속공제 요건 사후관리
2,가업승계 및 가업상속과 관련된 조세 지원제도는 어떻게 되나요?,가업승계 가업상속 관련 조세 지원제도
3,가업승계주식 증여특례 요건 및 그 사후관리가 궁금합니다.,가업승계주식 증여특례 요건 사후관리
4,가족 혹은 자녀 명의의 금융거래 또한 차명거래 금지 대상에 포함되나요?,가족명의 자녀명의 금융거래 차명거래 금지대상여부
5,간주 상속재산이 무엇인가요?,간주상속재산
6,보혐료 납입자와 보험금 수령자가 다를 때 계약자가 중도 인출을 하게 되면 어떻게 되나요?,계약자 수익자 다른경우 계약자 중도인출
7,보험료 납입자와 보험금 수령자가 다를 때 수익자가 보험금을 수령하게 되면 증여세 계...,계약자 수익자 다른경우 수익자 보험금수령 증여세 어떻 계산
8,보험료 납입자와 보험금 수령자가 동일하지 않을 때 증여세 부과는 언제인가요?,계약자 수익자 다른경우 증여세 언제부과
9,"계약자에게 해지할 권한이 없는 경우, 상속재산에 대한 평가가 어떻게 이루어지는 궁금...",계약자 해지권한 없는경우 상속재산평가


In [116]:
token = []
okt = Okt()
hannanum = Hannanum()
komoran = Komoran()

In [117]:
for i in range(len(data.values)):
    token.append(" ".join(okt.nouns(list(data.values[i])[0])))
data["twitter_token"] = token
token = []
for i in range(len(data.values)):
    token.append(" ".join(hannanum.nouns(list(data.values[i])[0])))
data["hannanum_token"] = token
token = []
for i in range(len(data.values)):
    token.append(" ".join(komoran.nouns(list(data.values[i])[0])))
data["komoran_token"] = token

In [119]:
data.to_excel("new_questions.xlsx", index = False)

In [121]:
#파라미터 0 뉴 데이터
data = get_complex(data, "hannanum")
data = get_complex(data, "twitter")
data = get_complex(data, "komoran")

data = get_complex_plus(data, "hannanum")
data = get_complex_plus(data, "twitter")
data = get_complex_plus(data, "komoran")

data = get_denominator(data)

data = get_accuracy_origin(data, "hannanum")
data = get_accuracy_origin(data, "twitter")
data = get_accuracy_origin(data, "komoran")

data = get_accuracy_complex(data, "hannanum")
data = get_accuracy_complex(data, "twitter")
data = get_accuracy_complex(data, "komoran")

data = get_accuracy_plus(data, "hannanum")
data = get_accuracy_plus(data, "twitter")
data = get_accuracy_plus(data, "komoran")

data.to_csv("new_result.csv", encoding = 'utf-8-sig', index = False)
data.to_excel("new_result.xlsx", index = False)

It took  529.5802211761475 seconds.
It took  762.3625428676605 seconds.
It took  741.6052556037903 seconds.
It took  484.2537078857422 seconds.
It took  751.905707359314 seconds.
It took  637.8909463882446 seconds.


# 파라미터 1 기존 데이터

In [132]:
#파라미터 1 기존 데이터
data = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/토큰화/token_total.xlsx")
data = get_complex(data, "hannanum")
data = get_complex(data, "twitter")
data = get_complex(data, "komoran")

data = get_complex_plus(data, "hannanum")
data = get_complex_plus(data, "twitter")
data = get_complex_plus(data, "komoran")

data = get_denominator(data)

data = get_accuracy_origin(data, "hannanum")
data = get_accuracy_origin(data, "twitter")
data = get_accuracy_origin(data, "komoran")

data = get_accuracy_complex(data, "hannanum")
data = get_accuracy_complex(data, "twitter")
data = get_accuracy_complex(data, "komoran")

data = get_accuracy_plus(data, "hannanum")
data = get_accuracy_plus(data, "twitter")
data = get_accuracy_plus(data, "komoran")

data.to_csv("result_more_1.csv", encoding = 'utf-8-sig', index = False)
data.to_excel("result_more_1.xlsx", index = False)

It took  575.5988619327545 seconds.
It took  823.2618389129639 seconds.
It took  662.8237874507904 seconds.
It took  353.51968789100647 seconds.
It took  634.9336953163147 seconds.
It took  570.0154514312744 seconds.


# 파라미터 2 기존 데이터

In [135]:
#파라미터 1 기존 데이터
data = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/토큰화/token_total.xlsx")
data = get_complex(data, "hannanum")
data = get_complex(data, "twitter")
data = get_complex(data, "komoran")

data = get_complex_plus(data, "hannanum")
data = get_complex_plus(data, "twitter")
data = get_complex_plus(data, "komoran")

It took  546.0505840778351 seconds.
It took  807.5218513011932 seconds.
It took  748.5115633010864 seconds.
It took  384.15544748306274 seconds.
It took  660.303085565567 seconds.
It took  584.2488512992859 seconds.


In [144]:
data

,질문,answer,twitter_token,hannanum_token,komoran_token,hannanum_complex,twitter_complex,komoran_complex,hannanum_complex_plus,twitter_complex_plus,komoran_complex_plus
0,(계약자=수익자) 수익자가 법정상속인이 아닌 경우 계약자 사망으로 상속포기시 사망보...,계약자 수익자 수익자 법정상속인 아닌경우 계약자 사망 상속포기 사망보험금 수령 관계,계약 수익 수익 법정상속 경우 계약 사망 상속 포기 시 사망 보험금 수령 관계,(계약자=수익자) 수익자 법정상속인 경우 계약자 사망 상속포기시 사망보험금 수령 관계,계약자 수익자 수익자 법정 상속인 경우 계약자 사망 상속 포기 사망 보험금 수령 관계,,상속포기 포기시 시사망 사망보험금 보험금수령,법정상속인 상속포기 사망보험금 보험금수령,,상속포기시,
1,가업상속공제 요건 및 사후관리,가업상속공제 요건 사후관리,가업 상속 공제 요건 및 사후관리,가업상속공제 요 사후관리,가업 상속 공제 요건 사후 관리,,가업상속 상속공제 공제요건 요건및 및사후관리,가업상속 상속공제 공제요건 사후관리,,가업상속공제,가업상속공제
2,"가업승계, 가업상속과 관련된 조세 지원제도",가업승계 가업상속 관련 조세 지원제도,가업승계 가업 상속 관련 조세 지원 제도,가업승계 가업상속 관련 조세 지원제도,가업 승계 가업 상속 관련 조세 지원 제도,조세지원제도,가업상속 상속관련 조세지원 지원제도,가업승계 가업상속 상속관련 조세지원 지원제도,,조세지원제도,조세지원제도
3,가업승계주식 증여특례 요건 및 사후관리,가업승계주식 증여특례 요건 사후관리,가업승계 주식 증여 특례 요건 및 사후관리,가업승계주식 증여특례 요 사후관리,가업 승계 주식 증여 특례 요건 사후 관리,,주식증여 증여특례 특례요건 요건및 및사후관리,가업승계 주식증여 증여특례 특례요건 사후관리,,,
4,"가족, 자녀 명의의 금융거래도 차명거래 금지 대상 여부",가족명의 자녀명의 금융거래 차명거래 금지대상여부,가족 자녀 명의 금융 거래 차 거래 금지 대상 여부,가족 자녀 명의 금융거래 차명거래 금지 대상 여부,가족 자녀 명의 금융 거래 차명 거래 금지 대상 여부,자녀명의 차명거래금지 금지대상 대상여부,자녀명의 금융거래 거래차 차거래 거래금지 금지대상 대상여부,자녀명의 금융거래 차명거래 거래금지 금지대상 대상여부,,,차명거래금지
5,간주 상속재산이란,간주상속재산,간주 상속 재산,간주 상속재산,간주 상속 재산,간주상속재산,간주상속 상속재산,간주상속 상속재산,,간주상속재산,간주상속재산
6,계약자(보험료 납입자)와 수익자(보험금 수령자)가 다른 경우 계약자가 중도인출을 하...,계약자 수익자 다른경우 계약자 중도인출,계약 보험료 납 입자 수익 보험금 수령 다른 경우 계약 중도,계약자(보험료 납입자 수익자(보험금 수령자 경우 계약자 중도인출,계약자 보험료 납 입자 수익자 보험금 수령자 경우 계약자 중도 인출,,보험료납 납입자 보험금수령,보험료납 납입자 중도인출,,,
7,계약자(보험료 납입자)와 수익자(보험금 수령자)가 다른 경우 수익자가 보험금을 수령...,계약자 수익자 다른경우 수익자 보험금수령 증여세 어떻 계산,계약 보험료 납 입자 수익 보험금 수령 다른 경우 수익 보험금 수령 증여세 계산,계약자(보험료 납입자 수익자(보험금 수령자 경우 수익자 보험금 수령 증여세 계산,계약자 보험료 납 입자 수익자 보험금 수령자 경우 수익자 보험금 수령 증여세 계산,보험금수령,보험료납 납입자 보험금수령 보험금수령,보험료납 납입자 보험금수령,,,
8,계약자(보험료 납입자)와 수익자(보험금 수령자)가 다른 경우 증여세는 언제 부과되나요?,계약자 수익자 다른경우 증여세 언제부과,계약 보험료 납 입자 수익 보험금 수령 다른 경우 증여세 언제 부과,계약자(보험료 납입자 수익자(보험금 수령자 경우 증여세 부과,계약자 보험료 납 입자 수익자 보험금 수령자 경우 증여세 부과,증여세부과,보험료납 납입자 보험금수령,보험료납 납입자 증여세부과,,,
9,계약자에게 해지할 권한이 없는 경우 상속재산평가,계약자 해지권한 없는경우 상속재산평가,계약 해지 권한 경우 상속 재산 평가,계약자 해 권한 경우 상속재산평가,계약자 해지 권한 경우 상속 재산 평가,,계약해지 상속재산 재산평가,상속재산 재산평가,,,


In [148]:
data = get_denominator(data)
data = get_valid_denominator(data, "hannanum")
data = get_valid_denominator(data, "twitter")
data = get_valid_denominator(data, "komoran")

data = get_accuracy_origin(data, "hannanum")
data = get_accuracy_origin(data, "twitter")
data = get_accuracy_origin(data, "komoran")

data = get_accuracy_complex(data, "hannanum")
data = get_accuracy_complex(data, "twitter")
data = get_accuracy_complex(data, "komoran")

data = get_accuracy_plus(data, "hannanum")
data = get_accuracy_plus(data, "twitter")
data = get_accuracy_plus(data, "komoran")

data.to_csv("result_more_2.csv", encoding = 'utf-8-sig', index = False)
data.to_excel("result_more_2.xlsx", index = False)

In [151]:
#파라미터 3 기존 데이터
data = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/토큰화/token_total.xlsx")
data = get_complex(data, "hannanum")
data = get_complex(data, "twitter")
data = get_complex(data, "komoran")

data = get_complex_plus(data, "hannanum")
data = get_complex_plus(data, "twitter")
data = get_complex_plus(data, "komoran")
data = get_denominator(data)
data = get_valid_denominator(data, "hannanum")
data = get_valid_denominator(data, "twitter")
data = get_valid_denominator(data, "komoran")

data = get_accuracy_origin(data, "hannanum")
data = get_accuracy_origin(data, "twitter")
data = get_accuracy_origin(data, "komoran")

data = get_accuracy_complex(data, "hannanum")
data = get_accuracy_complex(data, "twitter")
data = get_accuracy_complex(data, "komoran")

data = get_accuracy_plus(data, "hannanum")
data = get_accuracy_plus(data, "twitter")
data = get_accuracy_plus(data, "komoran")

data.to_csv("result_more_3.csv", encoding = 'utf-8-sig', index = False)
data.to_excel("result_more_3.xlsx", index = False)

It took  488.5874972343445 seconds.
It took  672.2428421974182 seconds.
It took  684.2489995956421 seconds.
It took  378.6391501426697 seconds.
It took  628.9342396259308 seconds.
It took  552.6283333301544 seconds.


In [189]:
a = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/result_more_1_.xlsx")
a = get_valid_denominator(a, "hannanum")
a = get_valid_denominator(a, "twitter")
a = get_valid_denominator(a, "komoran")
a.to_excel("result_more_1__.xlsx")

In [190]:
b = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/result_more_0.xlsx")
b = get_valid_denominator(b, "hannanum")
b = get_valid_denominator(b, "twitter")
b = get_valid_denominator(b, "komoran")
b.to_excel("result_more_0__.xlsx")

In [173]:
a = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/result_more_1_.xlsx")
a = a["hannanum_complex"]
a = a.values
a

x_num = []
for i in range(len(a)):
    if a[i]:
        x_num.append(len(a[i].split()))
x_num


AttributeError: 'float' object has no attribute 'split'

In [ ]:
    x = data[tokener]
    x = x.values
    x_num = []
    for i in range(len(x)):
        x_num.append(len(x[i].split()))
    tokener = tokenizer[:3] + "_complex_num"
    data[tokener] = x_num
    
    tokener = tokenizer + "_complex_plus"
    x = data[tokener]
    x = x.values
    x_num = []
    for i in range(len(x)):
        x_num.append(len(x[i].split()))
    tokener = tokenizer[:3] + "_plus_num"
    data[tokener] = x_num
    return data

In [176]:
a = pd.read_excel("C:/Users/HoonCheol/workspace/New_Tokenizer/result_more_1_.xlsx")
a = a["hannanum_complex"]
a = a.values
type(a[14])

float